Load data from file CSV

In [1]:
import pandas as pd
users = pd.read_csv('../data/raw/users.csv')
campaigns = pd.read_csv('../data/raw/campaigns.csv')
ads = pd.read_csv('../data/raw/ads.csv')
ad_events = pd.read_csv('../data/raw/ad_events.csv')

1. DIM_USER

In [2]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10000 non-null  object
 1   user_gender  10000 non-null  object
 2   user_age     10000 non-null  int64 
 3   age_group    10000 non-null  object
 4   country      10000 non-null  object
 5   location     10000 non-null  object
 6   interests    10000 non-null  object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [3]:
# remove: user_age, location
dim_user = users[
    ['user_id', 'user_gender', 'age_group', 'country', 'interests']
]

2. DIM_CAMPAIGN

In [4]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campaign_id    50 non-null     int64  
 1   name           50 non-null     object 
 2   start_date     50 non-null     object 
 3   end_date       50 non-null     object 
 4   duration_days  50 non-null     int64  
 5   total_budget   50 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [5]:
# remove: duration_days (calculate in Power BI)
dim_campaign = campaigns[
    ['campaign_id', 'name', 'start_date', 'end_date', 'total_budget']
]

3. DIM_ADS

In [6]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ad_id             200 non-null    int64 
 1   campaign_id       200 non-null    int64 
 2   ad_platform       200 non-null    object
 3   ad_type           200 non-null    object
 4   target_gender     200 non-null    object
 5   target_age_group  200 non-null    object
 6   target_interests  200 non-null    object
dtypes: int64(2), object(5)
memory usage: 11.1+ KB


In [ ]:
# keep all columns
dim_ads = ads[
    ['ad_id', 'campaign_id', 'ad_platform', 'ad_type', 'target_gender', 'target_age_group', 'target_interests']
]

4. DIM_DATE

In [8]:
ad_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   event_id     400000 non-null  int64 
 1   ad_id        400000 non-null  int64 
 2   user_id      400000 non-null  object
 3   timestamp    400000 non-null  object
 4   day_of_week  400000 non-null  object
 5   time_of_day  400000 non-null  object
 6   event_type   400000 non-null  object
dtypes: int64(2), object(5)
memory usage: 21.4+ MB


In [9]:
ad_events['timestamp'] = pd.to_datetime(ad_events['timestamp'])

In [10]:
ad_events['date'] = ad_events['timestamp'].dt.date
ad_events['year'] = ad_events['timestamp'].dt.year
ad_events['month'] = ad_events['timestamp'].dt.month
ad_events['month_name'] = ad_events['timestamp'].dt.month_name()
ad_events['quarter'] = ad_events['timestamp'].dt.quarter
ad_events['week_of_year'] = ad_events['timestamp'].dt.isocalendar().week
ad_events['day'] = ad_events['timestamp'].dt.day
ad_events['hour'] = ad_events['timestamp'].dt.hour

In [11]:
dim_date = ad_events[[
    'date',
    'year',
    'quarter',
    'month',
    'month_name',
    'week_of_year',
    'day',
    'day_of_week',
    'hour',
    'time_of_day'
]].drop_duplicates().reset_index(drop=True)

In [12]:
dim_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2185 entries, 0 to 2184
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          2185 non-null   object
 1   year          2185 non-null   int32 
 2   quarter       2185 non-null   int32 
 3   month         2185 non-null   int32 
 4   month_name    2185 non-null   object
 5   week_of_year  2185 non-null   UInt32
 6   day           2185 non-null   int32 
 7   day_of_week   2185 non-null   object
 8   hour          2185 non-null   int32 
 9   time_of_day   2185 non-null   object
dtypes: UInt32(1), int32(5), object(4)
memory usage: 121.8+ KB


In [ ]:
# create date_key in YYYYMMDD format (primary key)
dim_date['date_key'] = pd.to_datetime(dim_date['date']).dt.strftime('%Y%m%d').astype(int)

In [14]:
dim_date = dim_date[[
    'date_key',
    'date',
    'year',
    'quarter',
    'month',
    'month_name',
    'week_of_year',
    'day',
    'day_of_week',
    'hour',
    'time_of_day'
]]

5. FACT_AD_EVENTS

In [15]:
ad_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   event_id      400000 non-null  int64         
 1   ad_id         400000 non-null  int64         
 2   user_id       400000 non-null  object        
 3   timestamp     400000 non-null  datetime64[ns]
 4   day_of_week   400000 non-null  object        
 5   time_of_day   400000 non-null  object        
 6   event_type    400000 non-null  object        
 7   date          400000 non-null  object        
 8   year          400000 non-null  int32         
 9   month         400000 non-null  int32         
 10  month_name    400000 non-null  object        
 11  quarter       400000 non-null  int32         
 12  week_of_year  400000 non-null  UInt32        
 13  day           400000 non-null  int32         
 14  hour          400000 non-null  int32         
dtypes: UInt32(1), dat

In [17]:
# keep: event grain (1 row = 1 event)
# remove: day_of_week, time_of_day (handled by DIM_DATE)
fact_ad_events = ad_events[
    ['event_id', 'ad_id', 'user_id', 'timestamp', 'event_type']
]

EXPORT DATA

In [18]:
dim_user.to_csv('../data/processed/dim_user.csv', index=False)
dim_campaign.to_csv('../data/processed/dim_campaign.csv', index=False)
dim_ads.to_csv('../data/processed/dim_ads.csv', index=False)
dim_date.to_csv('../data/processed/dim_date.csv', index=False)
fact_ad_events.to_csv('../data/processed/fact_ad_events.csv', index=False)